## YOLO model Testing on Videos


In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("/home/ai/Desktop/RABs/TruckLoadUnload/models/TruckOID100v8.pt")  
video_path = "/home/ai/Desktop/RABs/Videos/unloading_video.mp4"  
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    results = model(frame)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            confidence = box.conf[0].item()  # Confidence score
            class_id = int(box.cls[0].item())  # Class ID
            label = f"{model.names[class_id]} {confidence:.2f}"

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    # Show the frame (optional, comment if not needed)
    cv2.imshow("YOLOv11 Video Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()



## Frame Extraction from Videos


In [ ]:

import cv2
import os

def extract_frames(video_path, output_folder, frame_interval=3):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return
    
    frame_count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        
        # Save every 3rd frame
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1
        
        frame_count += 1
    
    cap.release()
    print(f"Extraction complete. {saved_count} frames saved in '{output_folder}'.")

# Example usage
video_path = "Videos/unloading_video.mp4"  # Change to your video file path
output_folder = "TruckLoadUnload/TruckData/extracted_data"  # Change to your desired output directory
extract_frames(video_path, output_folder)




## WEB Scrapping from The google

In [8]:
import os
import time
import requests
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse

def create_directory(directory):
    """Create directory if it doesn't exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

def download_image(url, folder_path, filename=None):
    """Download an image from a URL and save it to the specified folder"""
    try:
        # Add a user agent to mimic a browser
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        
        response = requests.get(url, headers=headers, stream=True, timeout=10)
        response.raise_for_status()
        
        # Get the content type to determine file extension
        content_type = response.headers.get('Content-Type', '')
        extension = '.jpg'  # Default extension
        if 'png' in content_type.lower():
            extension = '.png'
        elif 'jpeg' in content_type.lower() or 'jpg' in content_type.lower():
            extension = '.jpg'
        elif 'gif' in content_type.lower():
            extension = '.gif'
        elif 'webp' in content_type.lower():
            extension = '.webp'
        
        # Create a filename if not provided
        if not filename:
            filename = f"image_{int(time.time())}_{random.randint(1000, 9999)}{extension}"
        elif not filename.endswith(extension):
            filename += extension
            
        filepath = os.path.join(folder_path, filename)
        
        # Save the image
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    
        print(f"Downloaded: {filepath}")
        return filepath
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def extract_images_from_google_selenium(search_url):
    """Extract image URLs from Google Images search results using Selenium"""
    # Create folder for saving images
    output_folder = "extracted_data"
    create_directory(output_folder)
    
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    
    # User agent to mimic a real browser
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    chrome_options.add_argument(f"user-agent={user_agent}")
    
    try:
        # Initialize the Chrome driver
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        
        # Load the Google Images page
        print(f"Opening URL: {search_url}")
        driver.get(search_url)
        
        # Wait for the page to load (adjust timeout as needed)
        time.sleep(5)
        
        # Scroll down to load more images (optional)
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
        
        # Find image elements
        print("Searching for images...")
        
        # Method 1: Find image containers
        image_elements = driver.find_elements(By.CSS_SELECTOR, ".rg_i")
        
        if not image_elements:
            # Try alternative selector
            image_elements = driver.find_elements(By.CSS_SELECTOR, "img.rg_i, img.Q4LuWd")
        
        if not image_elements:
            # Try yet another selector
            image_elements = driver.find_elements(By.CSS_SELECTOR, "img")
            # Filter out small images that are likely UI elements
            image_elements = [img for img in image_elements if 
                              img.get_attribute("width") and int(img.get_attribute("width")) > 100]
        
        print(f"Found {len(image_elements)} image elements")
        
        # Collect image URLs
        image_urls = []
        
        # Method 1: Direct image sources
        for img in image_elements:
            try:
                # Click on the image to load the full-size version (optional)
                driver.execute_script("arguments[0].click();", img)
                time.sleep(1)
                
                # Get the image URL
                src = img.get_attribute("src")
                if src and src.startswith("http") and "google" not in src:
                    image_urls.append(src)
                
                # Alternative: Try to get the high-res image URL
                large_img = driver.find_elements(By.CSS_SELECTOR, ".n3VNCb, [jsname='HiaYvf']")
                if large_img:
                    src = large_img[0].get_attribute("src")
                    if src and src.startswith("http") and "google" not in src:
                        image_urls.append(src)
            except Exception as e:
                print(f"Error processing image element: {e}")
        
        # Method 2: Extract image URLs from page source
        page_source = driver.page_source
        # Look for patterns like "https://example.com/image.jpg"
        import re
        url_pattern = r'https?://[^"\']+\.(jpg|jpeg|png|webp|gif)'
        urls = re.findall(url_pattern, page_source)
        
        for match in re.finditer(url_pattern, page_source):
            url = match.group(0)
            if "google" not in url and url not in image_urls:
                image_urls.append(url)
        
        # If we still don't have images, try one more method
        if not image_urls:
            print("Trying alternative method to extract images...")
            try:
                # Click on the first thumbnail to open the image viewer
                thumbnails = driver.find_elements(By.CSS_SELECTOR, ".isv-r")
                if thumbnails:
                    driver.execute_script("arguments[0].click();", thumbnails[0])
                    time.sleep(2)
                    
                    # Get the current image URL
                    img_viewers = driver.find_elements(By.CSS_SELECTOR, ".tvh9oe, .n3VNCb")
                    if img_viewers:
                        img_url = img_viewers[0].get_attribute("src")
                        if img_url and img_url.startswith("http") and "google" not in img_url:
                            image_urls.append(img_url)
                            
                    # Navigate through the image viewer for more images
                    for _ in range(min(10, len(thumbnails)-1)):
                        next_button = driver.find_elements(By.CSS_SELECTOR, ".hOXScf, [aria-label='Next image']")
                        if next_button:
                            driver.execute_script("arguments[0].click();", next_button[0])
                            time.sleep(1)
                            img_viewers = driver.find_elements(By.CSS_SELECTOR, ".tvh9oe, .n3VNCb")
                            if img_viewers:
                                img_url = img_viewers[0].get_attribute("src")
                                if img_url and img_url.startswith("http") and "google" not in img_url:
                                    image_urls.append(img_url)
            except Exception as e:
                print(f"Error with alternative method: {e}")
        
        # Remove duplicates
        image_urls = list(set(image_urls))
        print(f"Found {len(image_urls)} unique image URLs")
        
        # Download each image
        downloaded_count = 0
        for i, img_url in enumerate(image_urls):
            try:
                filename = f"truck_back_new_{i+1}"
                result = download_image(img_url, output_folder, filename)
                if result:
                    downloaded_count += 1
                # Add a small delay to avoid being blocked
                time.sleep(1)
            except Exception as e:
                print(f"Error processing {img_url}: {e}")
        
        print(f"Downloaded {downloaded_count} images to {output_folder}")
        return downloaded_count
    
    except Exception as e:
        print(f"Error during Selenium processing: {e}")
        return 0
    
    finally:
        # Clean up
        try:
            driver.quit()
        except:
            pass

if __name__ == "__main__":
    google_search_url = "https://www.google.com/search?q=truck%20images%20back&hl=en&tbs=rimg:Cai7s0_1eTP6rYfzQiOfNxbbSsgIAwAIA2AIA4AIA&udm=2&cs=1&sa=X&ved=0CB4QuIIBahcKEwjozYDm3t6LAxUAAAAAHQAAAAAQBw&biw=1856&bih=932&dpr=1"
    extract_images_from_google_selenium(google_search_url)

Opening URL: https://www.google.com/search?q=truck%20images%20back&hl=en&tbs=rimg:Cai7s0_1eTP6rYfzQiOfNxbbSsgIAwAIA2AIA4AIA&udm=2&cs=1&sa=X&ved=0CB4QuIIBahcKEwjozYDm3t6LAxUAAAAAHQAAAAAQBw&biw=1856&bih=932&dpr=1
Searching for images...
Found 100 image elements
Found 161 unique image URLs
Downloaded: extracted_data/truck_back_new_1.jpg
Downloaded: extracted_data/truck_back_new_2.jpg
Downloaded: extracted_data/truck_back_new_3.jpg
Downloaded: extracted_data/truck_back_new_4.jpg
Downloaded: extracted_data/truck_back_new_5.jpg
Downloaded: extracted_data/truck_back_new_6.jpg
Downloaded: extracted_data/truck_back_new_7.jpg
Downloaded: extracted_data/truck_back_new_8.jpg
Downloaded: extracted_data/truck_back_new_9.jpg
Downloaded: extracted_data/truck_back_new_10.jpg
Downloaded: extracted_data/truck_back_new_11.jpg
Downloaded: extracted_data/truck_back_new_12.jpg
Downloaded: extracted_data/truck_back_new_13.jpg
Downloaded: extracted_data/truck_back_new_14.jpg
Downloaded: extracted_data/truck_ba